In [1]:
import numpy as np
from sklearn.metrics import normalized_mutual_info_score

import torch
import torch.nn.functional as F
import torch_geometric as tg
from torch_geometric.nn import DenseSAGEConv, dense_diff_pool, dense_mincut_pool
from torch_geometric.utils import to_dense_adj

from dataset import load_pyg_dataset

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda', 1)


import os
import pandas as pd
import os.path as osp
import math

In [2]:
def save_best_model(model, optimizer, epoch, outdir):
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        },
        os.path.join(outdir,
                     'best_model.pt'))

def load_best_model(model, outdir):
    checkpoint = torch.load(os.path.join(outdir, 'best_model.pt'))
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

In [ ]:
pooling = 'mincut' #(options: diffpool, mincut)
# dataset_name = 'Chameleon'



#Pooling selector
pooling_selector = {
    'diffpool': dense_diff_pool,
    'mincut': dense_mincut_pool
}




# 'WikiCS',
for dataset_name in ['Cornell', 'Texas', 'Wisconsin', "Chameleon", "Squirrel", "Crocodile", 'Actor',
                    'DeezerEurope', 'Cora', 'Pubmed', 'Citeseer', 
                    'Twitch-DE', 'Twitch-EN', 'Twitch-ES', 'Twitch-FR', 'Twitch-PT', 'Twitch-RU', 'WikiCS']:

    #Creates outdir
    outdir = 'results_node_clustering/{}_{}'.format(dataset_name, pooling)
    os.makedirs(outdir, exist_ok = True)

    #Loads dataset
    data = load_pyg_dataset(
        data_name=dataset_name,
        device=device
    )
    num_clusters = data.y.max().tolist()+1
    data.adj = to_dense_adj(data.edge_index)


    #GNN to compute transformed node features for pooling (for assignation matrix)
    class GNN(torch.nn.Module):
        def __init__(self, in_channels, hidden_channels, out_channels,
                     normalize=False, lin=True):
            super(GNN, self).__init__()
            self.conv1 = DenseSAGEConv(in_channels, hidden_channels, normalize)
            self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv2 = DenseSAGEConv(hidden_channels, hidden_channels, normalize)
            self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv3 = DenseSAGEConv(hidden_channels, out_channels, normalize)
            self.bn3 = torch.nn.BatchNorm1d(out_channels)
            if lin is True:
                self.lin = torch.nn.Linear(
                    2 * hidden_channels + out_channels,
                    out_channels
                )
            else:
                self.lin = None

        def bn(self, i, x):
            batch_size, num_nodes, num_channels = x.size()
            x = x.view(-1, num_channels)
            x = getattr(self, 'bn{}'.format(i))(x)
            x = x.view(batch_size, num_nodes, num_channels)
            return x

        def forward(self, x, adj, mask=None):
            x0 = x
            if pooling == 'diffpool':
                x1 = self.bn(1, F.relu(self.conv1(x0, adj, mask)))
                x2 = self.bn(2, F.relu(self.conv2(x1, adj, mask)))
                x3 = self.bn(3, F.relu(self.conv3(x2, adj, mask)))
            elif pooling == 'mincut':
                x1 = F.relu(self.conv1(x0, adj, mask))
                x2 = F.relu(self.conv2(x1, adj, mask))
                x3 = F.relu(self.conv3(x2, adj, mask))
            x = torch.cat([x1, x2, x3], dim=-1)
            if self.lin is not None:
                x = F.relu(self.lin(x))
            return x

    #Net to compute pooling in an unsupervised manner
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.gnn1_pool = GNN(data.num_node_features, 64, num_clusters)
            self.gnn1_embed = GNN(data.num_node_features, 64, 64, lin=False)
            self.pooling = pooling_selector[pooling]

        def forward(self, x, adj, mask=None):
            s = self.gnn1_pool(x, adj, mask)
            x = self.gnn1_embed(x, adj, mask)
            x, adj, l1, e1 = self.pooling(x, adj, s, mask)
            return torch.softmax(s, dim=-1), l1, e1, adj # returns assignation matrix, and auxiliary losses and new adj matrix




    ###########
    #Training
    ###########
    #Optimizer, model
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    #Trains using auxiliary losses only
    def train(epoch):
        model.train()
        optimizer.zero_grad()
        s, l1, e1, adj = model(data.x, data.adj)
        loss = l1 + e1
        loss.backward()
        optimizer.step()
        return loss

    #Measures mutual information of clustering computed by pooling and ground truth
    @torch.no_grad()
    def test():
        model.eval()
        pred_node_label = model(data.x, data.adj)[0].max(dim=-1)[1].detach().cpu().numpy() 
        truth_node_labels = data.y.cpu().numpy()
        nmi = normalized_mutual_info_score(truth_node_labels.flatten(), pred_node_label.flatten())
        return nmi



    log_handle = open(osp.join(outdir, 'log.txt'), 'w')
    best_val_acc = best_test_acc = 0
    for epoch in range(1, 251):
        train_loss = train(epoch)
        val_acc = test()
        test_acc = test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            save_best_model(model, optimizer, epoch, outdir)
        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val NMI: {val_acc:.4f}, Test NMI: {test_acc:.4f}')
        log_handle.write(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val NMI: {val_acc:.4f}, Test NMI: {test_acc:.4f}\n')

    print(f'Best Val NMI: {best_val_acc:.4f}, Test NMI: {best_test_acc:.4f}')
    log_handle.write(f'\nBest Val NMI: {best_val_acc:.4f}, Test NMI: {best_test_acc:.4f}\n')
    log_handle.close()



    #Loads best model
    model = load_best_model(model, outdir)
    model.eval()
    node_assignation = model(data.x, data.adj)[0].max(-1)[1].detach().cpu().numpy().squeeze()
    adj_matrix = model(data.x, data.adj)[3].detach().cpu().numpy().squeeze()


    #Computes node clustering and adj matrices and saves them
    pd.DataFrame(node_assignation).transpose().to_csv(osp.join(outdir, 'node_assignation.csv'), header = None, index=False)
    pd.DataFrame(adj_matrix).to_csv(osp.join(outdir, 'adj_matrix.csv'), header = None, index=False)



In [3]:
pooling = 'mincut' #(options: diffpool, mincut)
# dataset_name = 'Chameleon'



#Pooling selector
pooling_selector = {
    'diffpool': dense_diff_pool,
    'mincut': dense_mincut_pool
}




# 'WikiCS',
# for dataset_name in ['Cornell', 'Texas', 'Wisconsin', "Chameleon", "Squirrel", "Crocodile", 'Actor',
#                     'DeezerEurope', 'Cora', 'Pubmed', 'Citeseer', 
#                     'Twitch-DE', 'Twitch-EN', 'Twitch-ES', 'Twitch-FR', 'Twitch-PT', 'Twitch-RU', 'WikiCS']:


for dataset_name in ['Cornell', 'Texas', 'Wisconsin', "Chameleon", "Squirrel", "Crocodile", 'Actor',
                    'DeezerEurope', 'Cora', 'Pubmed', 'Citeseer', 
                    'Twitch-DE', 'Twitch-EN', 'Twitch-ES', 'Twitch-FR', 'Twitch-PT', 'Twitch-RU', 'WikiCS']:

    #Creates outdir
    outdir = 'results_node_clustering_5_perc/{}_{}'.format(dataset_name, pooling)
    os.makedirs(outdir, exist_ok = True)

    #Loads dataset
    data = load_pyg_dataset(
        data_name=dataset_name,
        device=device
    )
#     num_clusters = data.y.max().tolist()+1
    num_clusters = math.ceil(0.05 * data.num_nodes)
    data.adj = to_dense_adj(data.edge_index)


    #GNN to compute transformed node features for pooling (for assignation matrix)
    class GNN(torch.nn.Module):
        def __init__(self, in_channels, hidden_channels, out_channels,
                     normalize=False, lin=True):
            super(GNN, self).__init__()
            self.conv1 = DenseSAGEConv(in_channels, hidden_channels, normalize)
            self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv2 = DenseSAGEConv(hidden_channels, hidden_channels, normalize)
            self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
            self.conv3 = DenseSAGEConv(hidden_channels, out_channels, normalize)
            self.bn3 = torch.nn.BatchNorm1d(out_channels)
            if lin is True:
                self.lin = torch.nn.Linear(
                    2 * hidden_channels + out_channels,
                    out_channels
                )
            else:
                self.lin = None

        def bn(self, i, x):
            batch_size, num_nodes, num_channels = x.size()
            x = x.view(-1, num_channels)
            x = getattr(self, 'bn{}'.format(i))(x)
            x = x.view(batch_size, num_nodes, num_channels)
            return x

        def forward(self, x, adj, mask=None):
            x0 = x
            if pooling == 'diffpool':
                x1 = self.bn(1, F.relu(self.conv1(x0, adj, mask)))
                x2 = self.bn(2, F.relu(self.conv2(x1, adj, mask)))
                x3 = self.bn(3, F.relu(self.conv3(x2, adj, mask)))
            elif pooling == 'mincut':
                x1 = F.relu(self.conv1(x0, adj, mask))
                x2 = F.relu(self.conv2(x1, adj, mask))
                x3 = F.relu(self.conv3(x2, adj, mask))
            x = torch.cat([x1, x2, x3], dim=-1)
            if self.lin is not None:
                x = F.relu(self.lin(x))
            return x

    #Net to compute pooling in an unsupervised manner
    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.gnn1_pool = GNN(data.num_node_features, 64, num_clusters)
            self.gnn1_embed = GNN(data.num_node_features, 64, 64, lin=False)
            self.pooling = pooling_selector[pooling]

        def forward(self, x, adj, mask=None):
            s = self.gnn1_pool(x, adj, mask)
            x = self.gnn1_embed(x, adj, mask)
            x, adj, l1, e1 = self.pooling(x, adj, s, mask)
            return torch.softmax(s, dim=-1), l1, e1, adj # returns assignation matrix, and auxiliary losses and new adj matrix




    ###########
    #Training
    ###########
    #Optimizer, model
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    #Trains using auxiliary losses only
    def train(epoch):
        model.train()
        optimizer.zero_grad()
        s, l1, e1, adj = model(data.x, data.adj)
        loss = l1 + e1
        loss.backward()
        optimizer.step()
        return loss

    #Test is the validation loss
    @torch.no_grad()
    def test():
        model.eval()
        s, l1, e1, adj = model(data.x, data.adj)
        loss = l1 + e1
        return loss

    log_handle = open(osp.join(outdir, 'log.txt'), 'w')
    best_val_loss = best_test_loss = np.inf
    for epoch in range(1, 251):
        train_loss = train(epoch)
        val_loss = test()
        test_loss = test()
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_test_loss = test_loss
            save_best_model(model, optimizer, epoch, outdir)
        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val loss: {val_loss:.4f}, Test loss: {test_loss:.4f}')
        log_handle.write(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.8f}, '
              f'Val loss: {val_loss:.4f}, Test loss: {test_loss:.4f}\n')

    print(f'Best Val loss: {best_val_loss:.4f}, Test loss: {best_test_loss:.4f}')
    log_handle.write(f'\nBest Val loss: {best_val_loss:.4f}, Test loss: {best_test_loss:.4f}\n')
    log_handle.close()



    #Loads best model
    model = load_best_model(model, outdir)
    model.eval()
    node_assignation = model(data.x, data.adj)[0].max(-1)[1].detach().cpu().numpy().squeeze()
    adj_matrix = model(data.x, data.adj)[3].detach().cpu().numpy().squeeze()


    #Computes node clustering and adj matrices and saves them
    pd.DataFrame(node_assignation).transpose().to_csv(osp.join(outdir, 'node_assignation.csv'), header = None, index=False)
    pd.DataFrame(adj_matrix).to_csv(osp.join(outdir, 'adj_matrix.csv'), header = None, index=False)




The obtained data Cornell has 183 nodes, 298 edges, 1703 features, 5 labels, 
Epoch: 001, Train Loss: 0.16997731, Val loss: 0.1694, Test loss: 0.1694
Epoch: 002, Train Loss: 0.16943437, Val loss: 0.1688, Test loss: 0.1688
Epoch: 003, Train Loss: 0.16880250, Val loss: 0.1680, Test loss: 0.1680
Epoch: 004, Train Loss: 0.16803193, Val loss: 0.1669, Test loss: 0.1669
Epoch: 005, Train Loss: 0.16687810, Val loss: 0.1650, Test loss: 0.1650
Epoch: 006, Train Loss: 0.16500223, Val loss: 0.1618, Test loss: 0.1618
Epoch: 007, Train Loss: 0.16176486, Val loss: 0.1564, Test loss: 0.1564
Epoch: 008, Train Loss: 0.15638280, Val loss: 0.1481, Test loss: 0.1481
Epoch: 009, Train Loss: 0.14806867, Val loss: 0.1357, Test loss: 0.1357
Epoch: 010, Train Loss: 0.13572168, Val loss: 0.1182, Test loss: 0.1182
Epoch: 011, Train Loss: 0.11818099, Val loss: 0.0950, Test loss: 0.0950
Epoch: 012, Train Loss: 0.09495521, Val loss: 0.0665, Test loss: 0.0665
Epoch: 013, Train Loss: 0.06650436, Val loss: 0.0341, Test

Epoch: 111, Train Loss: -0.49225080, Val loss: -0.4917, Test loss: -0.4917
Epoch: 112, Train Loss: -0.49170339, Val loss: -0.4917, Test loss: -0.4917
Epoch: 113, Train Loss: -0.49165833, Val loss: -0.4923, Test loss: -0.4923
Epoch: 114, Train Loss: -0.49225581, Val loss: -0.4919, Test loss: -0.4919
Epoch: 115, Train Loss: -0.49192071, Val loss: -0.4918, Test loss: -0.4918
Epoch: 116, Train Loss: -0.49175072, Val loss: -0.4922, Test loss: -0.4922
Epoch: 117, Train Loss: -0.49223995, Val loss: -0.4921, Test loss: -0.4921
Epoch: 118, Train Loss: -0.49208617, Val loss: -0.4919, Test loss: -0.4919
Epoch: 119, Train Loss: -0.49187052, Val loss: -0.4922, Test loss: -0.4922
Epoch: 120, Train Loss: -0.49222004, Val loss: -0.4922, Test loss: -0.4922
Epoch: 121, Train Loss: -0.49220693, Val loss: -0.4920, Test loss: -0.4920
Epoch: 122, Train Loss: -0.49198258, Val loss: -0.4922, Test loss: -0.4922
Epoch: 123, Train Loss: -0.49220240, Val loss: -0.4923, Test loss: -0.4923
Epoch: 124, Train Loss: -

Epoch: 227, Train Loss: -0.49768841, Val loss: -0.4983, Test loss: -0.4983
Epoch: 228, Train Loss: -0.49828494, Val loss: -0.4989, Test loss: -0.4989
Epoch: 229, Train Loss: -0.49891472, Val loss: -0.4987, Test loss: -0.4987
Epoch: 230, Train Loss: -0.49873090, Val loss: -0.4984, Test loss: -0.4984
Epoch: 231, Train Loss: -0.49835253, Val loss: -0.4984, Test loss: -0.4984
Epoch: 232, Train Loss: -0.49843836, Val loss: -0.4987, Test loss: -0.4987
Epoch: 233, Train Loss: -0.49874806, Val loss: -0.4988, Test loss: -0.4988
Epoch: 234, Train Loss: -0.49876761, Val loss: -0.4987, Test loss: -0.4987
Epoch: 235, Train Loss: -0.49868071, Val loss: -0.4988, Test loss: -0.4988
Epoch: 236, Train Loss: -0.49877739, Val loss: -0.4989, Test loss: -0.4989
Epoch: 237, Train Loss: -0.49890065, Val loss: -0.4988, Test loss: -0.4988
Epoch: 238, Train Loss: -0.49884391, Val loss: -0.4988, Test loss: -0.4988
Epoch: 239, Train Loss: -0.49879229, Val loss: -0.4989, Test loss: -0.4989
Epoch: 240, Train Loss: -

Epoch: 086, Train Loss: -0.37628877, Val loss: -0.3763, Test loss: -0.3763
Epoch: 087, Train Loss: -0.37627757, Val loss: -0.3762, Test loss: -0.3762
Epoch: 088, Train Loss: -0.37623882, Val loss: -0.3762, Test loss: -0.3762
Epoch: 089, Train Loss: -0.37615621, Val loss: -0.3760, Test loss: -0.3760
Epoch: 090, Train Loss: -0.37601244, Val loss: -0.3759, Test loss: -0.3759
Epoch: 091, Train Loss: -0.37585485, Val loss: -0.3758, Test loss: -0.3758
Epoch: 092, Train Loss: -0.37584507, Val loss: -0.3761, Test loss: -0.3761
Epoch: 093, Train Loss: -0.37608707, Val loss: -0.3764, Test loss: -0.3764
Epoch: 094, Train Loss: -0.37637568, Val loss: -0.3764, Test loss: -0.3764
Epoch: 095, Train Loss: -0.37642801, Val loss: -0.3763, Test loss: -0.3763
Epoch: 096, Train Loss: -0.37626028, Val loss: -0.3761, Test loss: -0.3761
Epoch: 097, Train Loss: -0.37613571, Val loss: -0.3762, Test loss: -0.3762
Epoch: 098, Train Loss: -0.37624633, Val loss: -0.3764, Test loss: -0.3764
Epoch: 099, Train Loss: -

Epoch: 199, Train Loss: -0.37563753, Val loss: -0.3756, Test loss: -0.3756
Epoch: 200, Train Loss: -0.37562335, Val loss: -0.3760, Test loss: -0.3760
Epoch: 201, Train Loss: -0.37602043, Val loss: -0.3765, Test loss: -0.3765
Epoch: 202, Train Loss: -0.37650895, Val loss: -0.3767, Test loss: -0.3767
Epoch: 203, Train Loss: -0.37672925, Val loss: -0.3766, Test loss: -0.3766
Epoch: 204, Train Loss: -0.37656403, Val loss: -0.3763, Test loss: -0.3763
Epoch: 205, Train Loss: -0.37626600, Val loss: -0.3762, Test loss: -0.3762
Epoch: 206, Train Loss: -0.37619579, Val loss: -0.3764, Test loss: -0.3764
Epoch: 207, Train Loss: -0.37640727, Val loss: -0.3767, Test loss: -0.3767
Epoch: 208, Train Loss: -0.37667668, Val loss: -0.3767, Test loss: -0.3767
Epoch: 209, Train Loss: -0.37671208, Val loss: -0.3765, Test loss: -0.3765
Epoch: 210, Train Loss: -0.37654221, Val loss: -0.3764, Test loss: -0.3764
Epoch: 211, Train Loss: -0.37641549, Val loss: -0.3765, Test loss: -0.3765
Epoch: 212, Train Loss: -

...

Epoch: 217, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 218, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 219, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 220, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 221, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 222, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 223, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 224, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 225, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 226, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 227, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 228, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 229, Train Loss: 0.34010309, Val loss: 0.3401, Test loss: 0.3401
Epoch: 230, Train Loss: 0.34010309, Val loss: 0.3401, Test loss:

Epoch: 080, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 081, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 082, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 083, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 084, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 085, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 086, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 087, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 088, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 089, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 090, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 091, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 092, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 093, Train Loss: 0.36570841, Val loss: 0.3657, Test loss:

Epoch: 195, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 196, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 197, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 198, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 199, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 200, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 201, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 202, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 203, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 204, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 205, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 206, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 207, Train Loss: 0.36570841, Val loss: 0.3657, Test loss: 0.3657
Epoch: 208, Train Loss: 0.36570841, Val loss: 0.3657, Test loss:

Epoch: 058, Train Loss: 0.28955257, Val loss: 0.2845, Test loss: 0.2845
Epoch: 059, Train Loss: 0.28453362, Val loss: 0.2775, Test loss: 0.2775
Epoch: 060, Train Loss: 0.27746630, Val loss: 0.2726, Test loss: 0.2726
Epoch: 061, Train Loss: 0.27260852, Val loss: 0.2712, Test loss: 0.2712
Epoch: 062, Train Loss: 0.27123058, Val loss: 0.2698, Test loss: 0.2698
Epoch: 063, Train Loss: 0.26984751, Val loss: 0.2612, Test loss: 0.2612
Epoch: 064, Train Loss: 0.26120365, Val loss: 0.2577, Test loss: 0.2577
Epoch: 065, Train Loss: 0.25771070, Val loss: 0.2613, Test loss: 0.2613
Epoch: 066, Train Loss: 0.26128483, Val loss: 0.2505, Test loss: 0.2505
Epoch: 067, Train Loss: 0.25051785, Val loss: 0.2475, Test loss: 0.2475
Epoch: 068, Train Loss: 0.24752879, Val loss: 0.2503, Test loss: 0.2503
Epoch: 069, Train Loss: 0.25029421, Val loss: 0.2409, Test loss: 0.2409
Epoch: 070, Train Loss: 0.24088848, Val loss: 0.2384, Test loss: 0.2384
Epoch: 071, Train Loss: 0.23842692, Val loss: 0.2404, Test loss:

Epoch: 172, Train Loss: 0.12971222, Val loss: 0.1290, Test loss: 0.1290
Epoch: 173, Train Loss: 0.12901604, Val loss: 0.1288, Test loss: 0.1288
Epoch: 174, Train Loss: 0.12877047, Val loss: 0.1295, Test loss: 0.1295
Epoch: 175, Train Loss: 0.12954962, Val loss: 0.1298, Test loss: 0.1298
Epoch: 176, Train Loss: 0.12980330, Val loss: 0.1308, Test loss: 0.1308
Epoch: 177, Train Loss: 0.13078451, Val loss: 0.1308, Test loss: 0.1308
Epoch: 178, Train Loss: 0.13082993, Val loss: 0.1300, Test loss: 0.1300
Epoch: 179, Train Loss: 0.13002491, Val loss: 0.1283, Test loss: 0.1283
Epoch: 180, Train Loss: 0.12826931, Val loss: 0.1268, Test loss: 0.1268
Epoch: 181, Train Loss: 0.12682617, Val loss: 0.1259, Test loss: 0.1259
Epoch: 182, Train Loss: 0.12589538, Val loss: 0.1257, Test loss: 0.1257
Epoch: 183, Train Loss: 0.12567902, Val loss: 0.1262, Test loss: 0.1262
Epoch: 184, Train Loss: 0.12622631, Val loss: 0.1275, Test loss: 0.1275
Epoch: 185, Train Loss: 0.12749970, Val loss: 0.1295, Test loss:

In [ ]:
len(set(node_assignation))

In [ ]:
model(data.x, data.adj)[0]

In [ ]:
num_clusters

In [ ]:
math.ceil(0.05 * data.num_nodes)

In [ ]:
data.num_nodes

In [ ]:
27 86